#### SQLite port_lite database: orders table
#### csv files: orders, ORDERS-PLACED

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 3, 2)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market'.split()
colt = 'trans name qty target active spd current change percent reason market'.split()

In [3]:
format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

In [4]:
pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market
0,25,B,JASIF,"10,000",11.20,2,DOS,SET


In [5]:
tmp = pd.read_csv('../data/orders-log.csv')
tmp['trans'] = tmp.apply(lambda row: categorise(row), axis=1)
tmp['amount'] = tmp.qty * tmp.target
tmp[tmp.active == 2].style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
3,B,JASIF,-1,DOS,SET,"10,000",11.20,11.30,0.10,+0.89%,2,Buy,"112,000.00"
6,B,TMT,-5,ROUND,SET,"15,000",10.50,11.00,0.00,0.00%,2,Buy,"157,500.00"
7,B,TPIPL,-4,nan,SET,"60,000",1.70,1.74,0.01,+0.58%,2,Buy,"102,000.00"
10,S,CPNCG,4,DOS,SET,"10,000",12.70,12.30,-0.10,-0.81%,2,Sell,"127,000.00"
17,S,SINGER,5,20pct,SET100,"1,500",50.75,49.50,1.25,+2.59%,2,Sell,"76,125.00"
19,S,SNC,0,DOS,SET,"9,000",17.10,17.10,0.00,0.00%,2,Sell,"153,900.00"
21,S,WHART,5,DOS,SET,"10,000",12.80,12.30,0.00,0.00%,2,Sell,"128,000.00"


In [6]:
tmp.head().sort_values('spd',ascending=False).style.hide_index().format(format_dict)

trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
B,JASIF,-1,DOS,SET,"10,000",11.20,11.30,0.10,+0.89%,2,Buy,"112,000.00"
B,GLOBAL,-6,nan,SET50,"9,000",19.40,20.00,0.30,+1.52%,1,Buy,"174,600.00"
B,ASP,-13,3L,SET,"30,000",3.60,3.86,-0.02,-0.52%,0,Buy,"108,000.00"
B,BAM,-13,nan,SET100,"10,000",20.50,21.80,-0.10,-0.46%,0,Buy,"205,000.00"
B,MAKRO,-14,RD05pct,SET,"1,500",39.75,43.25,0.50,+1.17%,0,Buy,"59,625.00"


### End of Tables in the process

### Print to verify before upload file

In [75]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=   trade    name  ..., width=800)

In [76]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of transactional process

In [77]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_act_2 = pd.read_sql(sql, conlite)
df_act_2.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,HREIT,"10,000",8.95,"89,500.00",DOS,SET,2
1,B,KCE,"1,000",58.00,"58,000.00",RD20pct,SET50,2
2,B,TMT,"15,000",10.50,"157,500.00",ROUND,SET,2
3,B,TPIPL,"60,000",1.70,"102,000.00",,SET,2
4,S,CPNCG,"20,000",12.50,"250,000.00",DOS,SET,2
5,S,JASIF,"10,000",11.20,"112,000.00",DOS,SET,2
6,S,SINGER,"1,500",50.75,"76,125.00",20pct,SET100,2
7,S,SNC,"7,200",17.10,"123,120.00",DOS,SET,2
8,S,WHART,"10,000",12.80,"128,000.00",DOS,SET,2


In [78]:
orders_grp = df_act_2.groupby(['trade'])
orders_grp['amount'].sum()

trade
B    407000.0
S    689245.0
Name: amount, dtype: float64

In [79]:
mask = df_act_2.trade == 'B'
buy_stocks = df_act_2[mask]
buy_stocks.style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,HREIT,"10,000",8.95,"89,500.00",DOS,SET,2
1,B,KCE,"1,000",58.00,"58,000.00",RD20pct,SET50,2
2,B,TMT,"15,000",10.50,"157,500.00",ROUND,SET,2
3,B,TPIPL,"60,000",1.70,"102,000.00",,SET,2


In [80]:
cash = 1_263_000
buy_amt = buy_stocks.amount.sum()
balance = cash - buy_amt 
cash, buy_amt, balance

(1263000, 407000.0, 856000.0)

In [81]:
mask = df_act_2.trade == 'S'
sell_stocks = df_act_2[mask]
sell_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
4,S,CPNCG,"20,000",12.50,"250,000.00",DOS,SET,2
5,S,JASIF,"10,000",11.20,"112,000.00",DOS,SET,2
6,S,SINGER,"1,500",50.75,"76,125.00",20pct,SET100,2
7,S,SNC,"7,200",17.10,"123,120.00",DOS,SET,2
8,S,WHART,"10,000",12.80,"128,000.00",DOS,SET,2


In [82]:
sell_amt = round(sell_stocks.amount.sum(),2)
balance = cash + sell_amt 
(cash, sell_amt, balance)

(1263000, 689245.0, 1952245.0)

### After call ord-log (must call everytime that orders change)

In [83]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)
df['amount'] = df.qty * df.target

In [84]:
show(df[['trans','name','spd','reason','qty','target','amount','current','active']])

trans,name,spd,reason,qty,target,amount,current,active


In [85]:
mb = (df.trans == 'Buy') & (df.spd >= -5)
df[mb].sort_values(['name'],ascending=[True]).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
2,B,GLOBAL,-5,nan,SET50,"9,000",19.40,19.90,-0.10,-0.50%,1,Buy,"174,600.00"
3,B,HREIT,-2,DOS,SET,"10,000",8.95,9.05,0.10,+1.12%,2,Buy,"89,500.00"
4,B,KCE,-1,RD20pct,SET50,"1,000",58.00,58.25,-2.50,-4.12%,2,Buy,"58,000.00"
7,B,TMT,-5,ROUND,SET,"15,000",10.50,11.00,0.00,0.00%,2,Buy,"157,500.00"
8,B,TPIPL,-5,nan,SET,"60,000",1.70,1.75,0.01,+0.57%,2,Buy,"102,000.00"
9,B,TU,-3,nan,SET50,"9,000",19.80,20.10,-0.10,-0.50%,1,Buy,"178,200.00"


In [86]:
show(df[['trans','name','spd','reason','qty','target','current','percent','active']])

trans,name,spd,reason,qty,target,current,percent,active


In [87]:
ms = (df.trans == 'Sell') & (df.spd <= 6)
df[ms].sort_values(['name'],ascending=[True]).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
11,S,CPNCG,2,DOS,SET,"20,000",12.50,12.30,0.00,0.00%,2,Sell,"250,000.00"
14,S,JASIF,0,DOS,SET,"10,000",11.20,11.20,-0.10,-0.88%,2,Sell,"112,000.00"
17,S,PTT,6,5pct,SET50,"3,000",41.25,39.75,0.75,+1.92%,1,Sell,"123,750.00"
19,S,SINGER,6,20pct,SET100,"1,500",50.75,49.25,-0.25,-0.51%,2,Sell,"76,125.00"
21,S,SNC,2,DOS,SET,"7,200",17.10,16.90,-0.20,-1.17%,2,Sell,"123,120.00"
22,S,WHART,4,DOS,SET,"10,000",12.80,12.40,0.10,+0.81%,2,Sell,"128,000.00"


In [88]:
df[mb | ms].sort_values(['trade','name'],ascending=[True,True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
2,B,GLOBAL,-5,NaN,SET50,9000,19.40,19.90,-0.10,-0.50%,1,Buy,174600.0
3,B,HREIT,-2,DOS,SET,10000,8.95,9.05,0.10,+1.12%,2,Buy,89500.0
4,B,KCE,-1,RD20pct,SET50,1000,58.00,58.25,-2.50,-4.12%,2,Buy,58000.0
7,B,TMT,-5,ROUND,SET,15000,10.50,11.00,0.00,0.00%,2,Buy,157500.0
8,B,TPIPL,-5,NaN,SET,60000,1.70,1.75,0.01,+0.57%,2,Buy,102000.0
9,B,TU,-3,NaN,SET50,9000,19.80,20.10,-0.10,-0.50%,1,Buy,178200.0
11,S,CPNCG,2,DOS,SET,20000,12.50,12.30,0.00,0.00%,2,Sell,250000.0
14,S,JASIF,0,DOS,SET,10000,11.20,11.20,-0.10,-0.88%,2,Sell,112000.0
17,S,PTT,6,5pct,SET50,3000,41.25,39.75,0.75,+1.92%,1,Sell,123750.0
19,S,SINGER,6,20pct,SET100,1500,50.75,49.25,-0.25,-0.51%,2,Sell,76125.0


In [89]:
df[mb | ms].shape[0]

12

In [90]:
mask = (df.trade == 'B')
df[mb | ms].nlargest(5, "amount")[["name","trans","amount"]].style.format(format_dict)

,name,trans,amount
11,CPNCG,Sell,"250,000.00"
9,TU,Buy,"178,200.00"
2,GLOBAL,Buy,"174,600.00"
7,TMT,Buy,"157,500.00"
22,WHART,Sell,"128,000.00"


### Select source of orders between these two

In [91]:
df_out = df[mb | ms][colt]
df_out.style.format(format_dict)

,trans,name,qty,target,active,spd,current,change,percent,reason,market
2,Buy,GLOBAL,"9,000",19.40,1,-5,19.90,-0.10,-0.50%,nan,SET50
3,Buy,HREIT,"10,000",8.95,2,-2,9.05,0.10,+1.12%,DOS,SET
4,Buy,KCE,"1,000",58.00,2,-1,58.25,-2.50,-4.12%,RD20pct,SET50
7,Buy,TMT,"15,000",10.50,2,-5,11.00,0.00,0.00%,ROUND,SET
8,Buy,TPIPL,"60,000",1.70,2,-5,1.75,0.01,+0.57%,nan,SET
9,Buy,TU,"9,000",19.80,1,-3,20.10,-0.10,-0.50%,nan,SET50
11,Sell,CPNCG,"20,000",12.50,2,2,12.30,0.00,0.00%,DOS,SET
14,Sell,JASIF,"10,000",11.20,2,0,11.20,-0.10,-0.88%,DOS,SET
17,Sell,PTT,"3,000",41.25,1,6,39.75,0.75,+1.92%,5pct,SET50
19,Sell,SINGER,"1,500",50.75,2,6,49.25,-0.25,-0.51%,20pct,SET100


In [92]:
df_out = df[df.active == 2][colt]
df_out.style.format(format_dict)

,trans,name,qty,target,active,spd,current,change,percent,reason,market
3,Buy,HREIT,"10,000",8.95,2,-2,9.05,0.10,+1.12%,DOS,SET
4,Buy,KCE,"1,000",58.00,2,-1,58.25,-2.50,-4.12%,RD20pct,SET50
7,Buy,TMT,"15,000",10.50,2,-5,11.00,0.00,0.00%,ROUND,SET
8,Buy,TPIPL,"60,000",1.70,2,-5,1.75,0.01,+0.57%,nan,SET
11,Sell,CPNCG,"20,000",12.50,2,2,12.30,0.00,0.00%,DOS,SET
14,Sell,JASIF,"10,000",11.20,2,0,11.20,-0.10,-0.88%,DOS,SET
19,Sell,SINGER,"1,500",50.75,2,6,49.25,-0.25,-0.51%,20pct,SET100
21,Sell,SNC,"7,200",17.10,2,2,16.90,-0.20,-1.17%,DOS,SET
22,Sell,WHART,"10,000",12.80,2,4,12.40,0.10,+0.81%,DOS,SET


In [93]:
file_name = 'ORDERS-PLACED.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### End of Output ORDERS-PLACED

In [ ]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
WHERE ACTIVE = 2
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

### Evening Process

In [ ]:
new_active = 1
old_active = 2

In [ ]:
sql = """
UPDATE orders 
SET active = %s
WHERE active = %s
"""
sql = sql % (new_active, old_active)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
names = df_out['name']
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

In [ ]:
sql = """
UPDATE orders 
SET active = 2
WHERE name IN (%s)
"""
sql = sql % (in_p)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
#To force orders.csv and orders table to be same copy
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of Evening Process